<a href="https://colab.research.google.com/github/PurviChaurasia/company-shortcoming-using-tweets/blob/main/Team3_acm_project_code_roBERTa_ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Collection

In [ ]:
!pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "lang:en #Myntra"

tweets = []
limit = 5000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():

    if len(tweets) == limit:
        break
    else:
        promotion_users = ["Myntra", "BigIndianDeals", "offersinday", "24x7deals1"] # on manual evaluation of tweets it is found these accounts are offer accounts and are hence not relevant to the scope of this project
        if tweet.user.username in promotion_users:
            continue
        elif len(tweet.content) == 1:
            continue
        else:
            tweets.append([tweet.date, tweet.user.username, tweet.content])


df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)
df.to_csv(r"D:\ACM\Data\Myntra\myntra.csv", index=False)

#Sentiment Analysis using roBERTa

In [ ]:
!pip install transformers
!pip install scipy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd

In [ ]:
df = pd.read_csv("Myntra.csv")

In [ ]:
df

,Unnamed: 0,Date,User,Tweet,tokens
0,0,2022-07-13 11:53:05+00:00,mirayu05,some days i purchased nike from myntra which w...,"['some', 'days', 'i', 'purchased', 'nike', 'fr..."
1,1,2022-07-13 11:44:23+00:00,iMahindroo,myntra so its sorted that my money is stolen i...,"['myntra', 'so', 'its', 'sorted', 'that', 'my'..."
2,2,2022-07-13 11:40:14+00:00,TarunPulsar180,myntra these sku's are under day express deli...,"['myntra', 'these', ""sku's"", 'are', 'under', '..."
3,3,2022-07-13 11:34:47+00:00,Arunabha_Gupta,myntra myntra another extremely poor quality c...,"['myntra', 'myntra', 'another', 'extremely', '..."
4,4,2022-07-13 11:34:18+00:00,Muskan_patel_06,myntra now the delivery guy is denying to come...,"['myntra', 'now', 'the', 'delivery', 'guy', 'i..."
...,...,...,...,...,...
2695,2695,2022-05-05 16:47:25+00:00,aasthasingh1446,rd attempt of delivering and pick-up of the pr...,"['rd', 'attempt', 'of', 'delivering', 'and', '..."
2696,2696,2022-05-05 16:22:12+00:00,santanujoshi,myntra myntrasupport amazonin meeshoofficial l...,"['myntra', 'myntrasupport', 'amazonin', 'meesh..."
2697,2697,2022-05-05 15:48:30+00:00,santanujoshi,myntra myntrasupport amazon reliancedigital me...,"['myntra', 'myntrasupport', 'amazon', 'relianc..."
2698,2698,2022-05-05 14:47:42+00:00,RupayTrading,myntrasupport myntra very bad delivery support,"['myntrasupport', 'myntra', 'very', 'bad', 'de..."


In [ ]:
 #Loading the model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(roberta)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
labels = ["Negetive", "Neutral", "Positive"]

In [ ]:
import numpy as np

In [ ]:
tweets = df['Tweet']
df1 = df
df1["sentiment"] = np.nan

for i in range(len(tweets)):
    tweet_words = []
    for word in tweets[i].split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = 'http'
        tweet_words.append(word)
            
    tweet_proc = ' '.join(tweet_words)
    encoded_tweet = tokenizer(tweet_proc, return_tensors = 'pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    convert = softmax(scores)
    for j in range(len(scores)):
        l = labels[j]
        s = convert[j]
        if s == max(convert):
            df1["sentiment"][i] = l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df1

,Unnamed: 0,Date,User,Tweet,tokens,sentiment
0,0,2022-07-13 11:53:05+00:00,mirayu05,some days i purchased nike from myntra which w...,"['some', 'days', 'i', 'purchased', 'nike', 'fr...",Negetive
1,1,2022-07-13 11:44:23+00:00,iMahindroo,myntra so its sorted that my money is stolen i...,"['myntra', 'so', 'its', 'sorted', 'that', 'my'...",Negetive
2,2,2022-07-13 11:40:14+00:00,TarunPulsar180,myntra these sku's are under day express deli...,"['myntra', 'these', ""sku's"", 'are', 'under', '...",Neutral
3,3,2022-07-13 11:34:47+00:00,Arunabha_Gupta,myntra myntra another extremely poor quality c...,"['myntra', 'myntra', 'another', 'extremely', '...",Negetive
4,4,2022-07-13 11:34:18+00:00,Muskan_patel_06,myntra now the delivery guy is denying to come...,"['myntra', 'now', 'the', 'delivery', 'guy', 'i...",Negetive
...,...,...,...,...,...,...
2695,2695,2022-05-05 16:47:25+00:00,aasthasingh1446,rd attempt of delivering and pick-up of the pr...,"['rd', 'attempt', 'of', 'delivering', 'and', '...",Negetive
2696,2696,2022-05-05 16:22:12+00:00,santanujoshi,myntra myntrasupport amazonin meeshoofficial l...,"['myntra', 'myntrasupport', 'amazonin', 'meesh...",Negetive
2697,2697,2022-05-05 15:48:30+00:00,santanujoshi,myntra myntrasupport amazon reliancedigital me...,"['myntra', 'myntrasupport', 'amazon', 'relianc...",Negetive
2698,2698,2022-05-05 14:47:42+00:00,RupayTrading,myntrasupport myntra very bad delivery support,"['myntrasupport', 'myntra', 'very', 'bad', 'de...",Negetive


# Aspect term extraction and sentiment classification via [PyABSA](https://github.com/yangheng95/PyABSA)
More usages see [here](https://github.com/yangheng95/PyABSA/tree/release/demos/aspect_term_extraction)

In [ ]:
!pip install pyabsa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 248 kB 8.2 MB/s 
     |████████████████████████████████| 43 kB 415 kB/s 
     |████████████████████████████████| 4.7 MB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 58.7 MB/s 
     |████████████████████████████████| 181 kB 47.3 MB/s 
     |████████████████████████████████| 314 kB 58.4 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pytorch-warmup: filename=pytorch_warmup-0.1.0-py3-none-any.whl size=5802 sha256=7b06f4c318f21a24e82a5f7fbb7f3efce3475c74af800a151e0948bb51b102a6
  Stored in directory: /root/.cache/pip/wheels/c7/5d/24/9475e442daa1e9332c122c79fb5131b9e4e91946009365902

## Find Available Checkpoints For **Current Version**

In [ ]:
from pyabsa import available_checkpoints
checkpoint_map = available_checkpoints(from_local=False)

/usr/local/lib/python3.7/dist-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


This script could only be used to manage NVIDIA GPUs,but no GPU found in your device
Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: None
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets
There may be some checkpoints available for early versions of PyABSA, see ./checkpoints.json


# Init a aspect extractor from online checkpoint or local checkpoint

In [ ]:
from pyabsa import ABSADatasetList, available_checkpoints
from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                   auto_device=True  # False means load model on CPU
                                   )


# Aspect Term Extract & Sentiment Inference

In [ ]:
tweets = list(df["Tweet"])

In [ ]:
l = [tweets[0]]

In [ ]:
l

['some days i purchased nike from myntra which was just according to the photos no damage but the fit and size was not comfortable we requested exchange for the same the delivery guy came and declared the product as damaged now no return no exchange no refund']

In [ ]:
# You can inference from a list of setences or a DatasetItem from PyABSA 
examples = ['myntra so its sorted that my money is stolen in the delivery fraud you have supported since the date of delivery #myntrafraud no reply to mails no proof given of delivery absolute low of service industry help me with escalations please myntrasupport']
inference_source = [tweets[0]]
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          save_result=True,
                          print_result=True,  # print the result
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

atepc_result[0]['sentiment']


/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:539: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bucket_pos = np.where(abs_pos <= mid, relative_pos, log_pos * sign).astype(np.int)


The results of aspect term extraction have been saved in /content/atepc_inference.result.json
Example 0: some days i purchased nike from myntra which was just according to the photos no damage but the <fit:Negative Confidence:0.9990091323852539> and <size:Negative Confidence:0.9988170862197876> was not comfortable we requested exchange for the same the <delivery:Negative Confidence:0.9995872378349304> guy came and declared the product as damaged now no <return:Negative Confidence:0.9971166849136353> no exchange no refund


/usr/local/lib/python3.7/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:408: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]


['Negative', 'Negative', 'Negative', 'Negative']

#Running it on Myntra's dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Myntra.csv")

In [ ]:
df["aspects"] = np.nan
df["sentiments"] = np.nan
tweets = list(df["Tweet"])
for i in range(len(tweets)):
    inference_source = [tweets[i]]
    atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          save_result=True,
                          print_result=True,  # print the result
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )
    df["aspects"][i] = atepc_result[0]['aspect']
    df["sentiments"][i] = atepc_result[0]['sentiment']

/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:539: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bucket_pos = np.where(abs_pos <= mid, relative_pos, log_pos * sign).astype(np.int)
/usr/local/lib/python3.7/dist-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:408: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]
/usr/local/lib/python3.7/dist-pack

Streaming output truncated to the last 5000 lines.
The results of aspect term extraction have been saved in /content/atepc_inference.result.json
Example 0: myntrasupport : my wife has ordered gown and she don ' t like that she had requested for return but return request keeps on cancelling from past week from <delivery:Negative Confidence:0.9997596144676208> agent this is the worst <service:Negative Confidence:0.9996280670166016> from myntra and its mental torture for me and my wife as she is months pregnant
The results of aspect term extraction have been saved in /content/atepc_inference.result.json
Example 0: myntra it is very serious condition the <delivery boy:Negative Confidence:0.9997802376747131> standing near my office thread me come and i will see youcall me <immideat:Neutral Confidence:0.9493847489356995> baseis
The results of aspect term extraction have been saved in /content/atepc_inference.result.json
Example 0: myntra same scripted responses # myntradeliveryfraud continue

In [ ]:
df

,Unnamed: 0,Date,User,Tweet,tokens,aspects,sentiments
0,0,2022-07-13 11:53:05+00:00,mirayu05,some days i purchased nike from myntra which w...,"['some', 'days', 'i', 'purchased', 'nike', 'fr...","[fit, size, delivery, return]","[Negative, Negative, Negative, Negative]"
1,1,2022-07-13 11:44:23+00:00,iMahindroo,myntra so its sorted that my money is stolen i...,"['myntra', 'so', 'its', 'sorted', 'that', 'my'...","[delivery, service]","[Negative, Negative]"
2,2,2022-07-13 11:40:14+00:00,TarunPulsar180,myntra these sku's are under day express deli...,"['myntra', 'these', ""sku's"", 'are', 'under', '...",[delivery],[Positive]
3,3,2022-07-13 11:34:47+00:00,Arunabha_Gupta,myntra myntra another extremely poor quality c...,"['myntra', 'myntra', 'another', 'extremely', '...","[service, delivery agent]","[Negative, Negative]"
4,4,2022-07-13 11:34:18+00:00,Muskan_patel_06,myntra now the delivery guy is denying to come...,"['myntra', 'now', 'the', 'delivery', 'guy', 'i...",[delivery guy],[Negative]
...,...,...,...,...,...,...,...
2695,2695,2022-05-05 16:47:25+00:00,aasthasingh1446,rd attempt of delivering and pick-up of the pr...,"['rd', 'attempt', 'of', 'delivering', 'and', '...",[delivering],[Negative]
2696,2696,2022-05-05 16:22:12+00:00,santanujoshi,myntra myntrasupport amazonin meeshoofficial l...,"['myntra', 'myntrasupport', 'amazonin', 'meesh...",[],[]
2697,2697,2022-05-05 15:48:30+00:00,santanujoshi,myntra myntrasupport amazon reliancedigital me...,"['myntra', 'myntrasupport', 'amazon', 'relianc...",[amazon],[Negative]
2698,2698,2022-05-05 14:47:42+00:00,RupayTrading,myntrasupport myntra very bad delivery support,"['myntrasupport', 'myntra', 'very', 'bad', 'de...",[delivery],[Negative]


In [ ]:
df.to_csv('final_out.csv')

# Deployment Example

In [ ]:
# -*- coding: utf-8 -*-
# file: deploy_demo.py
# time: 2021/10/10
# author: yangheng <yangheng@m.scnu.edu.cn>
# github: https://github.com/yangheng95
# Copyright (C) 2021. All Rights Reserved.

import gradio as gr
import pandas as pd

from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                                               auto_device=True  # False means load model on CPU
                                                               )


def inference(text):
    result = aspect_extractor.extract_aspect(inference_source=[text],
                                             pred_sentiment=True)

    result = pd.DataFrame({
        'aspect': result[0]['aspect'],
        'sentiment': result[0]['sentiment']
    })
    return result


if __name__ == '__main__':
    iface = gr.Interface(
        fn=inference,
        inputs=["text"],
        examples=[['The wine list is incredible and extensive and diverse , the food is all incredible and the staff was all very nice ,'
                   ' good at their jobs and cultured .'],
                  ['Though the menu includes some unorthodox offerings (a peanut butter roll, for instance), the classics are pure and '
                   'great--we have never had better sushi anywhere, including Japan.'],
                  ['Everything, from the soft bread, soggy salad, and 50 minute wait time, with an incredibly rude service to deliver'
                   ' below average food .'],
                  ['Even though it is running Snow Leopard, 2.4 GHz C2D is a bit of an antiquated CPU and thus the occasional spinning '
                   'wheel would appear when running Office Mac applications such as Word or Excel .'],
                  ['This demo is trained on the laptop and restaurant and other review datasets from ABSADatasets (https://github.com/yangheng95/ABSADatasets)'],
                  ['To fit on your data, please train the model on your own data, see the PyABSA (https://github.com/yangheng95/PyABSA)'],
                  ],
        outputs="dataframe",
        title='Aspect Term Extraction for Short Texts (powered by PyABSA)'
    )

    iface.launch()


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 6.8 MB/s 
     |████████████████████████████████| 272 kB 56.8 MB/s 
     |████████████████████████████████| 54 kB 611 kB/s 
     |████████████████████████████████| 141 kB 57.3 MB/s 
     |████████████████████████████████| 212 kB 58.4 MB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 2.3 MB 39.3 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 7.0 MB/s 
     |████████████████████████████████| 68 kB 6.7 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 4.1 MB 38.5 MB/s 
     |████████████████████████████████| 62 kB 1.1 

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa")
model = AutoModel.from_pretrained("yangheng/deberta-v3-base-absa")
# model = AutoModel.from_pretrained("yangheng/deberta-v3-large-absa")

inputs = tokenizer("good product especially video and audio quality fantastic.", return_tensors="pt")
outputs = model(**inputs)

Downloading:   0%|          | 0.00/368 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/785 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/701M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:539: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bucket_pos = np.where(abs_pos <= mid, relative_pos, log_pos * sign).astype(np.int)
